# Projeto de Ranking de políticos brasileiros V2


Esse projeto foi sugerido no linkedin.com com o tema política.  
Existem pelo menos dois rankings de políticos disponíveis na internet.  

O Legisla Brasil:
https://lnkd.in/ecRdMvbV

e o Ranking dos Políticos:
https://lnkd.in/eubz2hwK

O ponto principal é descobrir se existe grau de associação entre eles. "Ou seja, um candidato é bom e ruim a depender do ranking? Se sim, por que?"



## Checkpoints sugeridos:
### 1) Coleta de dados via web scrapping. Bibliotecas como Selenium e BeautifulSoup podem ajudar.

### 2) Análise exploratória: Gráficos de dispersão entre nota padronizada dos dois rankings. São correlacionados? Existem outliers?

### 3) Modelo explicativo: Regressão linear múltipla entre os 4 itens do ranking do Legisla Brasil e a nota do Ranking dos Políticos, por exemplo. Os 4 itens do Legisla explicam qual variabilidade do Ranking dos Políticos?

### 4) Discussão: Por que são diferentes? Por que são iguais? Existem viéses políticos? São apartidários? Qual sua opinião que justifique os resultados?

In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import numpy as np
import re
from unidecode import unidecode #retirar acentos
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep

### Lista do site Legisla Brasil: https://indice.legislabrasil.org/public/explorador

In [2]:
# formatando a janela do chrome que irá ser usada na pesquisa do selenium
options = Options()
options.add_argument('--headless') #utilizar quando não desejar que a página abra
options.add_argument('window-size=1000,800')

In [3]:
# configurando o webdriver e abrindo o navegador
navegador = webdriver.Chrome(options=options)

navegador.get('https://indice.legislabrasil.org/public/explorador')
sleep(1)
# para pegar o conteudo da página
pag1_source = BeautifulSoup(navegador.page_source)
contador = pag1_source.find_all('div', {'class', 'box_dep'} )
num_dep = len(list(contador))

In [4]:
#entrando na página de notas de cada candidato
lista_geral = []
for n in range(1, num_dep+1):
    bc = '//*[@id="dados"]/div/div[4]/div['+str(n)+']/a/img'
    click_dep = navegador.find_element(By.XPATH, bc)
    click_dep.click()
    
    #sleep(0.1)
    #pegando o html individual de cada candidato
    bs_indv_cand = BeautifulSoup(navegador.page_source)
    #sleep(0.1)
    
    string_notas = bs_indv_cand.find_all(string=re.compile("indicators"))
    string_notas

    lista_nota_deputado = []
    
    #loop para pegar as notas dos 17 quesitos e adiconar a lista
    for y in string_notas:
        #transformando a string em lista em ordem definida
        z = list(dict.fromkeys(y.split('  ')))
        z = z[18].split('[')
        
        lista_notas_ind = []
        for x in z:
            lista_notas_ind.append((x.replace('],', '').replace('".",','')).split(','))
        del lista_notas_ind[0]
        del lista_notas_ind[0]

        for x in lista_notas_ind:
             lista_nota_deputado.append(float(x[1]))

    lista_geral.append(lista_nota_deputado)
    navegador.back()
    

In [5]:
print(lista_geral)

[[0.9, 0.9, 0.9, 0.0, 0.0, 0.0, 10.0, 0.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9, 1.8, 0.0, 9.1], [3.6, 5.5, 5.5, 0.9, 0.9, 1.8, 10.0, 1.8, 2.7, 0.0, 1.8, 0.0, 0.0, 2.7, 3.6, 0.9, 6.4], [1.8, 2.7, 1.8, 0.0, 0.9, 0.9, 9.1, 3.6, 0.0, 0.0, 0.0, 0.9, 0.0, 2.7, 3.6, 0.0, 6.4], [0.9, 1.8, 1.8, 0.0, 0.9, 3.6, 10.0, 3.6, 0.9, 0.0, 2.7, 0.9, 0.0, 0.9, 5.5, 0.9, 5.5], [8.2, 5.5, 5.5, 5.5, 3.6, 10.0, 10.0, 2.7, 10.0, 0.9, 3.6, 0.0, 0.0, 7.3, 10.0, 4.5, 10.0], [1.8, 1.8, 1.8, 0.0, 0.9, 1.8, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8, 3.6, 0.9, 10.0], [1.8, 2.7, 2.7, 0.0, 0.9, 0.9, 9.1, 0.9, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8, 3.6, 0.9, 7.3], [0.9, 0.9, 0.9, 0.0, 0.9, 0.9, 3.6, 4.5, 0.0, 0.0, 3.6, 0.0, 0.0, 0.9, 1.8, 0.0, 9.1], [10.0, 3.6, 3.6, 0.0, 0.0, 0.9, 10.0, 10.0, 10.0, 0.0, 2.7, 0.9, 0.0, 10.0, 3.6, 0.9, 9.1], [1.8, 1.8, 1.8, 0.0, 0.9, 1.8, 9.1, 0.0, 0.0, 0.0, 7.3, 0.0, 0.9, 2.7, 6.4, 0.9, 7.3], [2.7, 3.6, 3.6, 1.8, 0.0, 1.8, 10.0, 4.5, 0.9, 0.0, 5.5, 0.0, 0.0, 2.7, 6.4, 0.0, 5.5], [0.9, 0.9, 0.9, 0.0, 0.0, 

In [57]:
from unidecode import unidecode
lista_candidatos = pag1_source.find_all('div', {'class': 'box_dep'})
nome_candidato = []

for i in lista_candidatos:
    nome_candidato.append(unidecode(i.contents[5].text.lower()))
print(nome_candidato)

['abilio santana', 'abou anni', 'acacio favacho', 'adolfo viana', 'adriana ventura', 'adriano do baldy', 'aecio neves', 'aelton freitas', 'afonso florence', 'afonso hamm', 'afonso motta', 'aguinaldo ribeiro', 'airton faleiro', 'aj albuquerque', 'alan rick', 'alceu moreira', 'alcides rodrigues', 'ale silva', 'alessandro molon', 'alex manente', 'alex santana', 'alexandre frota', 'alexandre leite', 'alexandre padilha', 'alexandre serfiotis', 'alexis fonteyne', 'alice portugal', 'aliel machado', 'aline gurgel', 'aline sleutjes', 'altineu cortes', 'aluisio mendes', 'amaro neto', 'andre abdon', 'andre de paula', 'andre ferreira', 'andre figueiredo', 'andre fufuca', 'andre janones', 'angela amin', 'anibal gomes', 'antonio brito', 'arlindo chinaglia', 'arnaldo jardim', 'aroldo martins', 'arthur lira', 'arthur oliveira maia', 'assis carvalho', 'atila lins', 'atila lira', 'augusto coutinho', 'aurea carolina', 'aureo ribeiro', 'bacelar', 'baleia rossi', 'benedita da silva', 'benes leocadio', 'bet

In [24]:
#Lista de colunas para o dataframe
nome_features = []
for n in range(3,20):
    a = bs_indv_cand.find_all('p')
    nome_features.append(a[n].text)
    
nome_features

['1. Projetos',
 '2. Protagonismo',
 '3. Relevância das Autorias',
 '4. Votos em separado apresentados',
 '5. Substituitivos apresentados',
 '6. Relatorias apresentadas',
 '7. Presença em Plenário',
 '8. Emendas de Plenário',
 '9. Solicitação de informações protocoladas',
 '10. Propostas de Fiscalização e Controle protocoladas',
 '11. Emendas Parlamentares',
 '12. Emendas de Medidas Provisórias',
 '13. Emendas de orçamento',
 '14. Projetos de autoria com status especial',
 '15. Cargos ocupados na legislatura',
 '16. Requerimentos de Audiência Pública',
 '17. Desvio da posição majoritária do partido em votações']

['abilio santana', 'abou anni', 'acacio favacho', 'adolfo viana', 'adriana ventura', 'adriano do baldy', 'aecio neves', 'aelton freitas', 'afonso florence', 'afonso hamm', 'afonso motta', 'aguinaldo ribeiro', 'airton faleiro', 'aj albuquerque', 'alan rick', 'alceu moreira', 'alcides rodrigues', 'ale silva', 'alessandro molon', 'alex manente', 'alex santana', 'alexandre frota', 'alexandre leite', 'alexandre padilha', 'alexandre serfiotis', 'alexis fonteyne', 'alice portugal', 'aliel machado', 'aline gurgel', 'aline sleutjes', 'altineu cortes', 'aluisio mendes', 'amaro neto', 'andre abdon', 'andre de paula', 'andre ferreira', 'andre figueiredo', 'andre fufuca', 'andre janones', 'angela amin', 'anibal gomes', 'antonio brito', 'arlindo chinaglia', 'arnaldo jardim', 'aroldo martins', 'arthur lira', 'arthur oliveira maia', 'assis carvalho', 'atila lins', 'atila lira', 'augusto coutinho', 'aurea carolina', 'aureo ribeiro', 'bacelar', 'baleia rossi', 'benedita da silva', 'benes leocadio', 'bet

In [111]:
df = pd.DataFrame(lista_geral)

In [112]:
df.insert(0, 'nome_candidato', nome_candidato)
df_T = df.transpose()
df_T.columns = df_T.iloc[0]
df_T = df_T.drop(df_T.index[0])
df_T = df_T.astype(float)

In [113]:
df_T

nome_candidato,abilio santana,abou anni,acacio favacho,adolfo viana,adriana ventura,adriano do baldy,aecio neves,aelton freitas,afonso florence,afonso hamm,...,wellington roberto,wilson santiago,wladimir garotinho,wolney queiroz,ze augusto nalin,ze carlos,ze neto,ze silva,ze vitor,zeca dirceu
0,0.9,3.6,1.8,0.9,8.2,1.8,1.8,0.9,10.0,1.8,...,1.8,5.5,2.7,4.5,0.9,10.0,6.4,3.6,5.5,7.3
1,0.9,5.5,2.7,1.8,5.5,1.8,2.7,0.9,3.6,1.8,...,0.9,6.4,3.6,3.6,0.9,0.9,4.5,5.5,8.2,1.8
2,0.9,5.5,1.8,1.8,5.5,1.8,2.7,0.9,3.6,1.8,...,0.9,6.4,2.7,3.6,0.9,0.9,4.5,5.5,9.1,1.8
3,0.0,0.9,0.0,0.0,5.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.9,0.0,0.0,0.0,0.0,0.9,0.9,0.9,0.0
4,0.0,0.9,0.9,0.9,3.6,0.9,0.9,0.9,0.0,0.9,...,0.9,0.9,0.9,0.9,0.0,0.0,0.9,1.8,1.8,0.9
5,0.0,1.8,0.9,3.6,10.0,1.8,0.9,0.9,0.9,1.8,...,0.9,0.9,1.8,0.9,0.0,0.9,3.6,4.5,4.5,0.9
6,10.0,10.0,9.1,10.0,10.0,10.0,9.1,3.6,10.0,9.1,...,7.3,10.0,6.4,10.0,0.9,9.1,10.0,10.0,9.1,9.1
7,0.9,1.8,3.6,3.6,2.7,0.0,0.9,4.5,10.0,0.0,...,10.0,1.8,0.0,10.0,0.0,2.7,2.7,2.7,7.3,3.6
8,0.0,2.7,0.0,0.9,10.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,2.7,2.7,0.0,10.0,4.5,1.8,5.5,7.3
9,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0
